# OaaS SDK Tutorial (Runnable)

This notebook demonstrates the simplified OaaS SDK using mock mode so it runs locally without external services. We use top-level `await` in code cells for async calls.

It also showcases accessor decorators like `@oaas.getter` for direct, persisted reads of typed fields.


In [2]:
# Core imports
from oaas_sdk2_py import oaas, OaasObject, OaasConfig
from pydantic import BaseModel
from typing import Dict, List, Any

print("OaaS tutorial environment ready: SDK imported.")


OaaS tutorial environment ready: SDK imported.


In [3]:
# Configure OaaS for local/mock mode (no external services)
config = OaasConfig(
    async_mode=True,
    mock_mode=True,
    oprc_partition_default=0,
    auto_commit=True,
)
oaas.configure(config)

# Quick health check
health = oaas.health_check()
print("Health:", health)


Health: {'healthy': True, 'issues': [], 'warnings': ['Global Oparaca instance not initialized', 'AutoSessionManager not initialized', 'No services registered'], 'info': []}


## Define a Service

We define a `Counter` service with persistent fields. Typed attributes automatically persist via the SDK's state descriptor system.


In [4]:
# Request model
class IncrementRequest(BaseModel):
    amount: int = 1

# Define service
@oaas.service("Counter", package="example")
class Counter(OaasObject):
    count: int = 0
    history: List[str] = []

    @oaas.method()
    async def increment(self, req: IncrementRequest) -> int:
        self.count += req.amount
        self.history.append(f"Incremented by {req.amount}")
        return self.count

    # Accessors: direct reads of persisted fields
    @oaas.getter("count")
    async def get_value(self) -> int:
        ...

    @oaas.getter("history")
    async def get_history(self) -> List[str]:
        ...

    @oaas.method()
    async def reset(self) -> bool:
        self.count = 0
        self.history.clear()
        return True


## Create a Local Object

For testing, we create a local instance of the `Counter` service. This avoids any external server or agent dependencies.


In [5]:
# Create a object for testing
counter = Counter.create()
print("Created Counter")
print("object_id:", getattr(counter, "object_id", None))


Created Counter
object_id: 749371763701246783


## Run Method Calls

Now we'll invoke a few methods using `await` and print the results and state. The SDK handles serialization automatically.


In [6]:
# Invoke methods with await
val = await counter.increment(IncrementRequest(amount=5))
print("increment ->", val)

cur = await counter.get_value()
print("get_value ->", cur)

hist = await counter.get_history()
print("get_history ->", hist)

ok = await counter.reset()
print("reset ->", ok)

cur2 = await counter.get_value()
print("after reset get_value ->", cur2)


increment -> 5
get_value -> 5
get_history -> ['Incremented by 5']
reset -> True
after reset get_value -> 0


## Advanced Service: More Methods and Types

We'll add a richer service that demonstrates:
- Returning multiple types (bool, str, bytes, dict, Pydantic model)
- Aggregated metadata
- Batch operations
- Reset and status helpers

Still using mock mode for local runs.

In [7]:
# Extra models and a richer service
from pydantic import BaseModel
from typing import Dict, List, Any

class SetValueRequest(BaseModel):
    value: int

class CounterInfo(BaseModel):
    value: int
    operations: int
    last_op: str | None

@oaas.service("CounterPlus", package="example")
class CounterPlus(OaasObject):
    count: int = 0
    history: List[str] = []
    metadata: Dict[str, Any] = {}

    @oaas.method()
    async def set_value(self, req: SetValueRequest) -> int:
        self.count = int(req.value)
        self.history.append(f"Set to {req.value}")
        return self.count

    @oaas.method()
    async def batch_increment(self, amounts: List[int]) -> List[int]:
        out: List[int] = []
        for a in amounts:
            self.count += a
            self.history.append(f"+{a}")
            out.append(self.count)
        return out

    @oaas.method()
    async def get_info(self) -> CounterInfo:
        return CounterInfo(value=self.count, operations=len(self.history), last_op=(self.history[-1] if self.history else None))

    @oaas.method()
    async def as_bytes(self) -> bytes:
        return str(self.count).encode()

    @oaas.method()
    async def is_positive(self) -> bool:
        return self.count > 0

    @oaas.method()
    async def get_status(self) -> str:
        return "positive" if self.count > 0 else ("zero" if self.count == 0 else "negative")

    @oaas.method()
    async def reset(self) -> bool:
        self.count = 0
        self.history.clear()
        self.metadata.clear()
        return True

INFO:oaas_sdk:Service CounterPlus successfully registered


### Try CounterPlus locally

We'll create a local object and exercise a few methods:
- set_value
- batch_increment
- get_info, is_positive, get_status
- as_bytes, reset

In [8]:
# Create and exercise CounterPlus
cp = CounterPlus.create()
print("Created CounterPlus")

print("set_value ->", await cp.set_value(SetValueRequest(value=3)))
print("batch_increment ->", await cp.batch_increment([2, 5, -4]))

info = await cp.get_info()
print("get_info ->", info)
print("is_positive ->", await cp.is_positive())
print("get_status ->", await cp.get_status())
print("as_bytes ->", await cp.as_bytes())

await cp.reset()
print("after reset get_info ->", await cp.get_info())

Created CounterPlus
set_value -> 3
batch_increment -> [5, 10, 6]
get_info -> value=6 operations=4 last_op='+-4'
is_positive -> True
get_status -> positive
as_bytes -> b'6'
after reset get_info -> value=0 operations=0 last_op=None


## Type System Quick Tour

Methods can return primitives, collections, bytes, and Pydantic models. We'll define a small service that returns various types so you can see serialization in action.

In [9]:
# A service demonstrating multiple return types
class DemoModel(BaseModel):
    id: int
    name: str

@oaas.service("DataTypesDemo", package="example")
class DataTypesDemo(OaasObject):
    iv: int = 42

    @oaas.method()
    async def return_int(self) -> int:
        return self.iv

    @oaas.method()
    async def return_float(self) -> float:
        return 3.1415

    @oaas.method()
    async def return_bool(self) -> bool:
        return True

    @oaas.method()
    async def return_str(self) -> str:
        return "hello"

    @oaas.method()
    async def return_list(self) -> List[int]:
        return [1, 2, 3]

    @oaas.method()
    async def return_dict(self) -> Dict[str, Any]:
        return {"k": "v", "n": 1}

    @oaas.method()
    async def return_bytes(self) -> bytes:
        return b"binary"

    @oaas.method()
    async def return_model(self) -> DemoModel:
        return DemoModel(id=1, name="demo")

INFO:oaas_sdk:Service DataTypesDemo successfully registered


In [10]:
# Exercise DataTypesDemo
demo = DataTypesDemo.create()
print("Created DataTypesDemo")
print("int ->", await demo.return_int())
print("float ->", await demo.return_float())
print("bool ->", await demo.return_bool())
print("str ->", await demo.return_str())
print("list ->", await demo.return_list())
print("dict ->", await demo.return_dict())
print("bytes ->", await demo.return_bytes())
print("model ->", await demo.return_model())

Created DataTypesDemo
int -> 42
float -> 3.1415
bool -> True
str -> hello
list -> [1, 2, 3]
dict -> {'k': 'v', 'n': 1}
bytes -> b'binary'
model -> id=1 name='demo'


Note: Avoid using attribute names reserved by the SDK (e.g., `meta`). Prefer user-defined names like `metadata` to store custom state.